# Laboratorio 9

In [ ]:
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt

from vega_datasets import data

alt.themes.enable('opaque')
%matplotlib inline

En este laboratorio utilizaremos un conjunto de datos _famoso_, el GapMinder. Esta es una versión reducida que solo considera países, ingresos, salud y población. 

¿Hay alguna forma natural de agrupar a estos países?

In [ ]:
gapminder = data.gapminder_health_income()
gapminder.head()

## Ejercicio 1

(1 pto.)

Realiza un Análisis exploratorio, como mínimo un `describe` del dataframe y una visualización adecuada, por ejemplo un _scatter matrix_ con los valores numéricos.

In [ ]:
gapminder.describe()

In [ ]:
base = alt.Chart(gapminder).mark_circle().encode(
    x=alt.X(alt.repeat("row"), type='quantitative'),
    y=alt.Y(alt.repeat("column"), type='quantitative'),
    tooltip=["country","income","health","population"]
).properties(
    width=300,
    height=300
)
scatter=base.repeat(
    row=["income","health","population"],
    column=["income","health","population"]
).interactive() 
scatter

__Pregunta:__ ¿Hay alguna variable que te entregue indicios a simple vista donde se puedan separar países en grupos?

__Respuesta:__  
* __TL:DR__ Se percibe un grupo para la mayoría de los gráficos, incluyendo analizar cada variable individualmente, con datos más dispersos en alguna dirección. Para el gráfico Salud vs Ingresos podría ser que hubieran más grupos , pero no sería tan fácil de percibir.
*  Para ingresos, notamos que los datos están concentrados asta alrededor de 45000, con una cantidad cada vez menor sobre esta marca.
* para el índice de salud, bajo app. 56, no hay tantos datos, después se distribuyen de manera pareja.
*  Para la población, la mayor parte esta concentrada asta alrededor de 10.000.000 con una cantidad menor sobre ese grupo.
* Hay claros outliers, como en el caso de la población estos outliers son China e india, en el caso de los ingresos es Qatar.
* para el gráfico Salud vs Población e ingresos vs Población, se percibe el efecto de la distribución de la población, con valores a lo largo de los intervalos de Salud, e ingresos, queson cada vez masdispersos al aumentar la población.
* Para el gráfico Salud vs Ingresos se agrupan entorno a una curva.
* Para el gráfico Salud vs Ingresos podría ser que hubieran más grupos , pero no sería tan fácil de percibir.

## Ejercicio 2

(1 pto.)

Aplicar un escalamiento a los datos antes de aplicar nuestro algoritmo de clustering. Para ello, definir la variable `X_raw` que corresponde a un `numpy.array` con los valores del dataframe `gapminder` en las columnas _income_, _health_ y  _population_. Luego, definir la variable `X` que deben ser los datos escalados de `X_raw`.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
X_raw = gapminder[["income","health","population"]]
scaler = StandardScaler()
X =  scaler.fit_transform(X_raw)

## Ejercicio 3

(1 pto.)

Definir un _estimator_ `KMeans` con `k=3` y `random_state=42`, luego ajustar con `X` y finalmente, agregar los _labels_ obtenidos a una nueva columna del dataframe `gapminder` llamada `cluster`. Finalmente, realizar el mismo gráfico del principio pero coloreado por los clusters obtenidos.


In [ ]:
from sklearn.cluster import KMeans

In [ ]:
k = 3
kmeans = KMeans(k,random_state=42)
kmeans.fit(X)
clusters = kmeans.labels_
gapminder["cluster"] = clusters

In [ ]:
base = alt.Chart(gapminder).mark_circle().encode(
    x=alt.X(alt.repeat("row"), type='quantitative'),
    y=alt.Y(alt.repeat("column"), type='quantitative'),
    color="cluster:N",
    tooltip=["country","income","health","population"]
).properties(
    width=300,
    height=300
)

base.repeat(
    row=["income","health","population"],
    column=["income","health","population"]
).interactive()


## Ejercicio 4

(1 pto.)

__Regla del codo__

__¿Cómo escoger la mejor cantidad de _clusters_?__

En este ejercicio hemos utilizado que el número de clusters es igual a 3. El ajuste del modelo siempre será mejor al aumentar el número de clusters, pero ello no significa que el número de clusters sea el apropiado. De hecho, si tenemos que ajustar $n$ puntos, claramente tomar $n$ clusters generaría un ajuste perfecto, pero no permitiría representar si existen realmente agrupaciones de datos.

Cuando no se conoce el número de clusters a priori, se utiliza la [regla del codo](https://jarroba.com/seleccion-del-numero-optimo-clusters/), que indica que el número más apropiado es aquel donde "cambia la pendiente" de decrecimiento de la la suma de las distancias a los clusters para cada punto, en función del número de clusters.

A continuación se provee el código para el caso de clustering sobre los datos estandarizados, leídos directamente de un archivo preparado especialmente.En la línea que se declara `kmeans` dentro del ciclo _for_ debes definir un estimador K-Means, con `k` clusters y `random_state` 42. Recuerda aprovechar de ajustar el modelo en una sola línea.

In [ ]:
elbow = pd.Series(name="inertia", dtype="float64").rename_axis(index="k")
for k in range(1, 10):
    kmeans = KMeans(k,random_state=42).fit(X)
    elbow.loc[k] = kmeans.inertia_ # Inertia: Sum of distances of samples to their closest cluster center
elbow = elbow.reset_index()

In [ ]:
alt.Chart(elbow).mark_line(point=True).encode(
    x="k:O",
    y="inertia:Q"
).properties(
    height=600,
    width=800
)

__Pregunta:__ Considerando los datos (países) y el gráfico anterior, ¿Cuántos clusters escogerías?

__Respuesta:__ Según la regla del codo sería k=4.Lo asociado por lo general serían 3 grupos de países, pero el metodo asocia un centroide a China e India, al ser Outliers de población. De esta manera, me quedaría con la clasificación que se da con 4 clusters, pero teniendo en cuenta la situación anterior.